In [2]:
import pandas as pd
import requests
import csv
import os
from tqdm import tqdm
import time

# Функция для получения района и координат из API 2ГИС
def get_district_and_coords(address, api_key):
    base_url = 'https://catalog.api.2gis.com/3.0/items'
    query = f"Алматы, {address}"
    params = {
        'q': query,
        'fields': 'items.adm_div,items.point',
        'key': api_key
    }

    try:
        response = requests.get(base_url, params=params, timeout=10)
        if response.status_code != 200:
            print(f"Ошибка запроса для адреса '{address}': статус {response.status_code}")
            return {'город': None, 'район': None, 'coodsX': None, 'coodY': None}

        data = response.json()
        items = data.get('result', {}).get('items', [])
        if not items:
            print(f"Нет данных для адреса '{address}'")
            return {'город': None, 'район': None, 'coodsX': None, 'coodY': None}

        district = None
        city = None
        lat = None
        lon = None

        for item in items:
            adm_div = item.get('adm_div', [])
            # Ищем в adm_div элемент с type == 'district'
            for adm in adm_div:
                if adm.get('type') == 'district':
                    district = adm.get('name')
                    break
            if district:
                # Получаем координаты
                point = item.get('point', {})
                lat = point.get('lat')
                lon = point.get('lon')
                # Получаем город
                for adm in adm_div:
                    if adm.get('type') == 'city':
                        city = adm.get('name')
                        break
                break

        print(f"Адрес: {address}")
        print(f"Город: {city}, Район: {district}, CoordsX: {lon}, CoodY: {lat}\n")

        return {'город': city, 'район': district, 'coodsX': lon, 'coodY': lat}

    except requests.RequestException as e:
        print(f"Ошибка запроса для адреса '{address}': {e}")
        return {'город': None, 'район': None, 'coodsX': None, 'coodY': None}
    except Exception as e:
        print(f"Неизвестная ошибка для адреса '{address}': {e}")
        return {'город': None, 'район': None, 'coodsX': None, 'coodY': None}

# Функция для записи данных в CSV
def write_to_csv(csv_file, header, row_data):
    file_exists = os.path.isfile(csv_file)
    with open(csv_file, mode='a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # Если файла не существует, пишем заголовок
        if not file_exists:
            writer.writerow(header)
        writer.writerow(row_data)

# Главная функция
def main():
    api_key = '56c7b3af-db16-445d-b2b3-30d87f605864'
    csv_file = 'processed_records.csv'
    
    # Считываем DataFrame
    df_filtered = pd.read_csv('/content/sample_data/filtered_data.csv', usecols=['channel_name', 'message', 'address'])

    # Устанавливаем стартовый индекс и максимальное количество записей для обработки
    start_index = 4335   # Задайте нужное стартовое значение
    max_records = 1000   # Задайте нужное количество записей. Можно использовать None для всех оставшихся
    
    # Если max_records задан, берем срез по указанным параметрам, иначе берем все с start_index до конца
    if max_records is not None:
        df_filtered = df_filtered.iloc[start_index:start_index+max_records].reset_index(drop=True)
    else:
        df_filtered = df_filtered.iloc[start_index:].reset_index(drop=True)

    # Проверяем, существует ли CSV-файл и считываем уже обработанные записи
    if os.path.isfile(csv_file):
        try:
            processed_df = pd.read_csv(csv_file)
            processed_set = set(zip(processed_df['channel_name'], processed_df['address']))
            print(f"Найдено {len(processed_set)} уже обработанных записей. Продолжаем с оставшихся.")
        except Exception as e:
            print(f"Ошибка при чтении существующего CSV-файла: {e}")
            processed_set = set()
    else:
        processed_set = set()
        print("Файл CSV не найден. Начинаем обработку с начала.")


    # Исключаем уже обработанные записи
    mask = df_filtered.apply(lambda row: (row['channel_name'], row['address']) not in processed_set, axis=1)
    df_to_process = df_filtered[mask].reset_index(drop=True)

    print(f"Осталось обработать {len(df_to_process)} записей.")

    if len(df_to_process) == 0:
        print("Все записи уже обработаны. Завершаем работу.")
        return

    # Определяем заголовок CSV (добавляем message)
    header = ['channel_name', 'message', 'address', 'город', 'район', 'coodsX', 'coodY']

    # Итерируемся по строкам DataFrame с прогресс-баром
    for index, row in tqdm(df_to_process.iterrows(), total=df_to_process.shape[0], desc="Обработка адресов"):
        channel_name = row['channel_name']
        message = row['message']
        address = row['address']

        # Получаем данные из API
        result = get_district_and_coords(address, api_key)

        # Формируем строку для записи
        row_data = [
            channel_name,
            message,
            address,
            result['город'] if result['город'] else '',
            result['район'] if result['район'] else '',
            result['coodsX'] if result['coodsX'] else '',
            result['coodY'] if result['coodY'] else ''
        ]

        # Записываем строку в CSV
        try:
            write_to_csv(csv_file, header, row_data)
            print(f"Запись добавлена: {row_data}")
        except Exception as e:
            print(f"Ошибка при записи в CSV для адреса '{address}': {e}")

        # Задержка между запросами (200 мс)
        time.sleep(0.2)

    print(f"\nВсе записи сохранены в файл '{csv_file}'.")

    # Вывод последней записи из CSV
    try:
        df_last = pd.read_csv(csv_file)
        if not df_last.empty:
            last_record = df_last.iloc[[-1]]
            print("\nПоследняя запись в CSV:")
            print(last_record)
        else:
            print("CSV-файл пуст.")
    except Exception as e:
        print(f"Ошибка при чтении CSV-файла для вывода последней записи: {e}")

# Запуск главной функции
if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/filtered_data.csv'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('processed_records.csv')
df.head()

ParserError: Error tokenizing data. C error: Expected 7 fields in line 1085, saw 13
